In [1]:
#! python -m pip install gekko

In [2]:
import numpy as np
import pandas as pd
from gekko import GEKKO

In [3]:
np.random.seed(1)
df_returns=pd.DataFrame(np.random.rand(100,4)-0.25,columns=list('ABCD'))
df_returns

,A,B,C,D
0,0.167022,0.470324,-0.249886,0.052333
1,-0.103244,-0.157661,-0.063740,0.095561
2,0.146767,0.288817,0.169195,0.435220
3,-0.045548,0.628117,-0.222612,0.420468
4,0.167305,0.308690,-0.109613,-0.051899
...,...,...,...,...
95,0.191904,0.331577,0.739752,-0.046094
96,-0.002267,0.012173,0.500172,0.206975
97,-0.193071,0.258516,-0.038040,0.548604
98,0.047331,-0.222394,0.343432,0.593840


In [4]:
weights=pd.Series([0.15,0.35,0.25,0.25],index=list('ABCD'))
weights

A    0.15
B    0.35
C    0.25
D    0.25
dtype: float64

In [5]:
0.167022*0.15,-0.103244*0.15

(0.0250533, -0.0154866)

In [6]:
df_returns.mean().values

array([0.24384783, 0.2789166 , 0.21977262, 0.29300383])

In [7]:
df_returns*weights

,A,B,C,D
0,0.025053,0.164614,-0.062471,0.013083
1,-0.015487,-0.055181,-0.015935,0.023890
2,0.022015,0.101086,0.042299,0.108805
3,-0.006832,0.219841,-0.055653,0.105117
4,0.025096,0.108041,-0.027403,-0.012975
...,...,...,...,...
95,0.028786,0.116052,0.184938,-0.011523
96,-0.000340,0.004261,0.125043,0.051744
97,-0.028961,0.090481,-0.009510,0.137151
98,0.007100,-0.077838,0.085858,0.148460


In [8]:
def returns_objective_function(x,df_returns):
    result=(df_returns*x).mean().sum()
    return result

In [9]:
x=np.ones(4)
x

array([1., 1., 1., 1.])

In [10]:
returns_objective_function(weights,df_returns)

0.2623920970760596

In [11]:
m=GEKKO()

In [12]:
x = m.Array(m.Var,4,value=0.01,lb=0.01,ub=0.75)
y=df_returns.mean().values

In [13]:
x

array([0.01, 0.01, 0.01, 0.01], dtype=object)

In [14]:
m.Equation(m.sum(x)==1)
m.Equation(x[0]+x[1]==0.3)

In [15]:
m.Maximize(m.sum([x[i]*y[i] for i in range(0,4)]))#subject function

In [16]:
m.solve()

apm 103.113.225.246_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            1
   Constants    :            0
   Variables    :            9
   Intermediates:            0
   Connections  :            5
   Equations    :            7
   Residuals    :            7
 
 Number of state variables:              9
 Number of total equations: -            7
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              2
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program c

In [17]:
for i in range(4):
    print ('Asset ',i+1,x[i].value[0])

Asset  1 0.01
Asset  2 0.29000000971
Asset  3 0.01
Asset  4 0.69000000998


In [18]:
print ('Objective: ' + str(-m.options.objfcnval))

Objective: 0.28769465995


In [19]:
from gekko import GEKKO
m = GEKKO(remote=False)
x = m.Array(m.Var,3,value=1,lb=0,ub=5)
# weights
p = [5,10,6.2]
# expected return on the asset
Rp = [0.1,0.05,0.3]
# risk free rate of return
Rf = [0.01,0.02,0.01]
# standard deviation of returns of the asset
sigma = [0.1,0.2,0.15]

# linear inequality constraint Ax<b
A = [[3, 6, 1], [8, 4, 3], [2, 2, 2]]
b = [30, 44, 35]
m.axb(A,b,x=x,etype='<')

# total budget
m.Equation(m.sum(x)<100)

# total assets purchased
ta = m.Intermediate(m.sum(x))

# maximize mean Sharpe ratio of portfolio
m.Maximize(m.sum([x[i]*(Rp[i]-Rf[i])/sigma[i] for i in range(3)])/ta)

m.options.solver = 1
m.solve(disp=True)
for i in range(3):
    print ('Asset ',i+1,x[i].value[0])
print ('Objective: ' + str(-m.options.objfcnval))

A
[[3. 6. 1.]
 [8. 4. 3.]
 [2. 2. 2.]]
b
[30. 44. 35.]
 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  2
   Constants    :  0
   Variables    :  8
   Intermediates:  1
   Connections  :  7
   Equations    :  6
   Residuals    :  5
 
 Number of state variables:    8
 Number of total equations: -  8
 Number of slack variables: -  1
 ---------------------------------------
 Degrees of freedom       :    -1
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0 -1.17589E+00  9.70000E+01
    1 -1.75141E+00  2.22045E-16
    2 -1.87018E+00  1.11022E-16
    3 -1.87018E+00  1.11022E-16
 Successful solution
 
 ----------------